In [ ]:
import pandas as pd
import os

# All Excel files
directory_path = "C:\\Users\\Piyush\\Downloads\\2024\\excel\\categorized\\Category_1\\before temp"

# Output path
output_path = 'C:\\Users\\Piyush\\Downloads\\2024\\excel\\categorized\\Category_1\\Temp cor'

os.makedirs(output_path, exist_ok=True)

# Loop through all Excel files in the directory
for file_name in os.listdir(output_path):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(directory_path, file_name)
        
        try:
            data_df = pd.read_excel(file_path)

            # Check if required columns are present
            if 'Time (second)' in data_df.columns and 'Time (sec) ' in data_df.columns and 'TC1 (°C)' in data_df.columns:
                time_voltage = data_df['Time (second)']
                time_temp = data_df['Time (sec) ']  # we have space here at the end

                # DataFrame with the voltage time
                unified_time = pd.DataFrame({'Time (second)': time_voltage})

                # Merge temperature data onto the unified time index using interpolation
                data_df.set_index('Time (sec) ', inplace=True)
                unified_temp = unified_time.merge(data_df[['TC1 (°C)']], left_on='Time (second)', right_index=True, how='left')

                # Interpolate missing temperature values based on the voltage time
                unified_temp['TC1 (°C)'] = unified_temp['TC1 (°C)'].interpolate(method='linear')

                # Replace the 'TC1 (°C)' column in the original DataFrame with the interpolated values
                data_df.reset_index(inplace=True)  
                data_df.set_index('Time (second)', inplace=True)
                data_df.update(unified_temp.set_index('Time (second)'))

            
                data_df.reset_index(inplace=True)

                # Reorder the columns to match the required structure
                required_columns = [
                    'Cell No', 'Time (second)', 'Load (lb)', 'Voltage (V)', 
                    'TC1 (°C)', 'TC2 (°C)', 'TC3 (°C)', 'TC4 (°C)', 
                    'TC5 (°C)', 'TC6 (°C)', 'Observed Score', 
                    'Observed Score ', 'Calculated Score', 'Capacity',
                    'SOC', 'A', 'B', 'C', 'max T', 'rate T', 'voltage Difference'
                ]
                data_df = data_df[required_columns]

                # Save the modified DataFrame to a new Excel file
                output_file_path = os.path.join(output_path, file_name)
                data_df.to_excel(output_file_path, index=False)

                print(f"{file_name} processed and saved")
            else:
                print(f"Skipping file {file_name} due to missing required columns.")
        
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

